# Aggregating streaming data from Spotify
We now have 2800 csv files with weekly streaming data from the top 200 artists on Spotify. In this notebook we will look over the csv's to retrieve the weekly sum of streams and save it per country/week. Finally we save this data in a pickle.

In [1]:
import os

import pandas as pd

In [2]:
base = os.path.join("..", "01.raw_data", "spotify")

def aggregate_csv_streams(country):
    """Returns list with sum of weekly streams per country."""
    
    # Get filenames in directory
    country_base = os.path.join(base, country)
    filenames = [filename for filename in os.listdir(country_base)]
    data = []
    
    # Import all the csv data
    for filename in filenames:
        # filename format 2018-08-24--2018-08-31.csv
        date = filename.split("--")[0]
        
        # We use a try-except because certain files seem bogus
        try:
            df = pd.read_csv(os.path.join(country_base, filename), header=1)
            data.append({"country": country,
                         "date": date,
                         "streams": df["Streams"].sum()})
        except pd.errors.ParserError: 
            print(f"{country} - {date}")
    
    return data

In [3]:
# For every country we take the data from the csv files. 
# The result will be a list of dictionaryies containing 
# country, week and num streams.
# We use the append method because a list comprehension 
# might use too much memory

data = []
for country in os.listdir(base):
    if (os.path.isdir(os.path.join(base, country)) 
            and not country.startswith(".")):
        data.extend(aggregate_csv_streams(country))

# Turns out there is no data for HR, MT and SI

hr - 2018-08-24
hr - 2018-08-31
hr - 2018-09-07
hr - 2018-09-14
hr - 2018-09-21
hr - 2018-09-28
hr - 2018-10-05
hr - 2018-10-12
hr - 2018-10-19
hr - 2018-10-26
hr - 2018-11-02
hr - 2018-11-09
hr - 2018-11-16
hr - 2018-11-23
hr - 2018-11-30
hr - 2018-12-07
hr - 2018-12-14
hr - 2018-12-21
hr - 2018-12-28
hr - 2019-01-04
hr - 2019-01-11
hr - 2019-01-18
hr - 2019-01-25
hr - 2019-02-01
hr - 2019-02-08
hr - 2019-02-15
hr - 2019-02-22
hr - 2019-03-01
hr - 2019-03-08
hr - 2019-03-15
hr - 2019-03-22
hr - 2019-03-29
hr - 2019-04-05
hr - 2019-04-12
hr - 2019-04-19
hr - 2019-04-26
hr - 2019-05-03
hr - 2019-05-10
hr - 2019-05-17
hr - 2019-05-24
hr - 2019-05-31
hr - 2019-06-07
hr - 2019-06-14
hr - 2019-06-21
hr - 2019-06-28
hr - 2019-07-05
hr - 2019-07-12
hr - 2019-07-19
hr - 2019-07-26
hr - 2019-08-02
hr - 2019-08-09
hr - 2019-08-16
hr - 2019-08-23
hr - 2019-08-30
hr - 2019-09-06
hr - 2019-09-13
hr - 2019-09-20
hr - 2019-09-27
hr - 2019-10-04
hr - 2019-10-11
hr - 2019-10-18
hr - 2019-10-25
hr - 201

In [4]:
# Check how much data we have now
len(data)

2600

In [5]:
# Save in dataframe and convert date type
df = pd.DataFrame(data)
df["date"] = pd.to_datetime(df["date"])
df.head()

,country,date,streams
0,at,2018-08-24,10301612
1,at,2018-08-31,9508616
2,at,2018-09-07,9717264
3,at,2018-09-14,9794815
4,at,2018-09-21,9570905


In [6]:
# 2 Lists to convert 2 letter codes to 3 letter codes
countries = "AUT, BEL, BGR, HRV, CYP, CZE, \
                DNK, EST, FIN, FRA, DEU, GRC, \
                HUN, IRL, ITA, LVA, LTU, LUX, \
                MLT, NLD, POL, PRT, ROU, SVK, \
                SVN, ESP, SWE, GBR"
iso3 = [c.strip() for c in countries.split(", ")]
iso2 = ['AT', 'BE', 'BG', 'HR', 'CY', 'CZ', 
        'DK', 'EE', 'FI', 'FR', 'DE', 'GR', 
        'HU', 'IE', 'IT', 'LV', 'LT', 'LU', 
        'MT', 'NL', 'PL', 'PT', 'RO', 'SK', 
        'SI', 'ES', 'SE', 'GB']
replacement = {k: v for k, v in zip(iso2, iso3)}

df["country"] = df["country"].str.upper().replace(replacement)
df.head()

,country,date,streams
0,AUT,2018-08-24,10301612
1,AUT,2018-08-31,9508616
2,AUT,2018-09-07,9717264
3,AUT,2018-09-14,9794815
4,AUT,2018-09-21,9570905


In [7]:
# Pivot the table to have the weeks as index and streams per country column
df = df.pivot(index="date", columns=["country"])

In [8]:
# Add a column with the mean of all countries per week
df["mean"] = df.mean(axis=1).astype(int)

In [9]:
# Export to pickle and csv
df.to_pickle("aggregated_country_streams.pkl")
df.to_csv("aggregated_country_streams.csv")

In [27]:
df.head()

streams                                                           \
country          AUT       BEL      BGR     CYP      CZE        DEU       DNK   
date                                                                            
2018-08-24  10301612  12348795   985485  328417  4869155  142899503  24716696   
2018-08-31   9508616  12634840  1136095  384379  5207132  148764686  25975199   
2018-09-07   9717264  12538824  1043073  340697  5010547  142763299  26251701   
2018-09-14   9794815  12994948  1011482  365756  5432395  151528074  26286642   
2018-09-21   9570905  12930082   971304  358192  5362867  148899467  25622860   

                                         ...                            \
country          ESP      EST       FIN  ...      LTU     LUX      LVA   
date                                     ...                             
2018-08-24  98020022  1438969  21122683  ...  1420563  736943  1507392   
2018-08-31  98106210  1498797  21368063  ...  1465384  796085  1629058   
2018-09-07  93060881  1435962  21121857  ...  1428749  775400  1602453   
2018-09-14  92130930  1402195  21057084  ...  1434227  800165  1603317   
2018-09-21  92773802  1377288  20912984  ...  1381935  812493  1558056   

                                                                         mean  
country          NLD       POL      PRT      ROU      SVK       SWE            
date                                                                           
2018-08-24  67114943  25430818  8730023  1958986  1745234  51913587  31920100  
2018-08-31  75928950  24843040  9444484  2391996  2024877  55020209  33699774  
2018-09-07  80271104  24352976  9183448  2108987  1975851  52755183  33092277  
2018-09-14  79581894  24416983  9055023  2152635  1982482  53277596  33763708  
2018-09-21  78606490  24437330  9494344  2138312  1969050  52795704  33819316  

[5 rows x 26 columns]